In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType
from pyspark.sql.functions import to_timestamp

In [2]:
# Spark Session
spark = SparkSession.builder.appName('Fraud-Detector').getOrCreate()

22/01/03 19:36:42 WARN Utils: Your hostname, francesc-Inspiron-5570 resolves to a loopback address: 127.0.1.1; using 192.168.1.141 instead (on interface wlp3s0)
22/01/03 19:36:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/03 19:36:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Read datasets
df_train = spark.read.csv('data/clean_fraudTrain.csv', header=True)
df_test = spark.read.csv('data/clean_fraudTest.csv', header=True)

In [4]:
# Show Pretty Data
df_train.limit(5).toPandas()

,timestamp,credit_card_num,shop,category,amount,gender,lat,long,city_pop,job,merch_lat,merch_long,is_fraud,age
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,0,31
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,0,41
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.262,4154,Nature conservation officer,43.150704,-112.154481,0,57
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.0,M,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,0,52
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,0,33


In [5]:
# Cast Data
df_train.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- credit_card_num: string (nullable = true)
 |-- shop: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city_pop: string (nullable = true)
 |-- job: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- age: string (nullable = true)



In [6]:
# Casting Train
df_train = df_train.drop("timestamp")
df_train = df_train.withColumn("credit_card_num",df_train.credit_card_num.cast(IntegerType()))
df_train = df_train.withColumn("amount",df_train.amount.cast(FloatType()))
df_train = df_train.withColumn("lat",df_train.lat.cast(FloatType()))
df_train = df_train.withColumn("long",df_train.long.cast(FloatType()))
df_train = df_train.withColumn("city_pop",df_train.city_pop.cast(IntegerType()))
df_train = df_train.withColumn("merch_lat",df_train.merch_lat.cast(FloatType()))
df_train = df_train.withColumn("merch_long",df_train.merch_long.cast(FloatType()))
df_train = df_train.withColumn("is_fraud",df_train.is_fraud.cast(IntegerType()))
df_train = df_train.withColumn("age",df_train.age.cast(IntegerType()))

# Casting Test
df_test = df_test.drop("timestamp")
df_test = df_test.withColumn("credit_card_num",df_test.credit_card_num.cast(IntegerType()))
df_test = df_test.withColumn("amount",df_test.amount.cast(FloatType()))
df_test = df_test.withColumn("lat",df_test.lat.cast(FloatType()))
df_test = df_test.withColumn("long",df_test.long.cast(FloatType()))
df_test = df_test.withColumn("city_pop",df_test.city_pop.cast(IntegerType()))
df_test = df_test.withColumn("merch_lat",df_test.merch_lat.cast(FloatType()))
df_test = df_test.withColumn("merch_long",df_test.merch_long.cast(FloatType()))
df_test = df_test.withColumn("is_fraud",df_test.is_fraud.cast(IntegerType()))
df_test = df_test.withColumn("age",df_test.age.cast(IntegerType()))
